In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
from sklearn.utils import shuffle

subset100 = pd.read_csv("../raw_data/track_meta_100subset_new.csv")
subset100 = shuffle(subset100)
# subset100 = subset100.reset_index()

### Train-val-test split

In [2]:
# features_drop = ["Playlist","Album", "Track", "Artist", "Trackid", "Artist_Name", "Track_Name", "Album_Name", "Artist_uri", "Track_uri", "Album_uri", "artist_genres", "explicit"]
# subset100_cleaned = subset100.drop(features_drop, axis =1) # remove playlist related features

In [3]:
# subset100_cleaned.dtypes

In [4]:
subset100.head()

,Playlistid,Trackid,Artist_Name,Track_Name,Album_Name,Track_Duration,Artist_uri,Track_uri,Album_uri,acousticness,...,loudness,mode,speechiness,tempo,time_signature,valence,Playlist,Album,Track,Artist
214,2535,81,Dj Slim D,I Got Two Phones,I Got Two Phones,241290,spotify:artist:5VRQkaqx6A28RnkJ8xx2nW,spotify:track:72ZxpfXEOPHMmqmYD0LpkR,spotify:album:2HYolsoIh0cUxObkDX4WNE,0.2150,...,-7.440,0,0.1350,121.764,4,0.1840,Feel Good,94,102,85
2972,256979,16,Bad Meets Evil,Lighters,Hell: The Sequel,303813,spotify:artist:77IURH5NC56Jn09QHi76is,spotify:track:4BWBbJ9MKpnt30SFYEomzJ,spotify:album:7CWSLPmiMIHXfETjpdOVT6,0.3510,...,-8.237,1,0.2650,90.079,4,0.1810,2011,43,54,40
1961,196206,6,AC/DC,T.N.T.,High Voltage,214666,spotify:artist:711MCceyCBcFnzjGY4Q7Un,spotify:track:7LRMbd3LEoV5wZJvXT1Lwb,spotify:album:19AUoKWRAaQYrggVvdQnqq,0.1530,...,-5.218,1,0.3020,126.367,4,0.4350,classic,13,17,8
858,62657,1,John Denver,"Take Me Home, Country Roads","The John Denver Collection, Vol. 1: Take Me Ho...",197813,spotify:artist:7EK1bQADBoqbYXnT4Cqv9w,spotify:track:2xYlyywNgefLCRDG8hlxZq,spotify:album:5erROp1lRW31aNxj9PbAUf,0.5460,...,-12.564,1,0.0322,164.267,4,0.5460,country road,17,19,16
1138,107941,7,Sia,Big Girls Cry,1000 Forms Of Fear,210613,spotify:artist:5WUlDfRSoLAfcVSX1WnrxN,spotify:track:75xgoXAGed7jiNDdV8TNOe,spotify:album:3xFSl9lIRaYXIYkIn3OIl9,0.0393,...,-4.962,1,0.0399,161.996,4,0.0938,Writing,9,9,9


In [5]:
# Train-val-test split (20%)
train, test = train_test_split(subset100, test_size=0.2, random_state=42, stratify = subset100['Playlistid'])
train, val = train_test_split(train, test_size=0.2, random_state=42, stratify = train['Playlistid'])

In [6]:
# Drop features here
features_drop = ["Playlistid","Playlist","Album", "Track", "Artist", "Trackid", "Artist_Name", "Track_Name", "Album_Name", "Artist_uri", "Track_uri", "Album_uri", "artist_genres", "explicit"]
train_cleaned, val_cleaned, test_cleaned = train.drop(features_drop, axis =1), val.drop(features_drop, axis=1), test.drop(features_drop, axis=1)

In [17]:
train = train.reset_index(drop=True)

In [7]:
train_cleaned = train_cleaned.reset_index(drop=True)

In [8]:
train_cleaned.head()

,Track_Duration,acousticness,artist_popularity,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,227880,0.00357,100,0.773,0.404,0.000022,6,0.0718,-7.438,0,0.0949,99.999,4,0.287
1,227960,0.01110,76,0.693,0.547,0.000000,1,0.0677,-8.127,1,0.1140,80.028,4,0.612
2,181933,0.41400,78,0.668,0.613,0.000000,4,0.1020,-4.225,1,0.1020,99.852,4,0.498
3,195199,0.01270,74,0.514,0.858,0.052800,2,0.2510,-5.995,1,0.0896,89.833,4,0.600
4,326280,0.10300,81,0.795,0.596,0.000000,11,0.1640,-3.799,0,0.2190,101.985,4,0.357


### Content-based Collaborative Filtering

In [12]:
# train_content
from sklearn.metrics.pairwise import cosine_similarity 

### Create a cosine-similarity matrix

In [10]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(train_cleaned)
train_scaled = scaler.transform(train_cleaned)
test_scaled = scaler.transform(test_cleaned)

In [13]:
train_scaled_cos_matrix = cosine_similarity(train_scaled)
# np.dot(np.array(trial_scaled[0,]), np.array(trial_scaled[5,]))/ (np.linalg.norm(trial_scaled[0,]) * np.linalg.norm(trial_scaled[5,]))


In [14]:
train_scaled_cos_matrix[:5]

array([[1.        , 0.81131319, 0.83194573, ..., 0.81758748, 0.86867039,
        0.86253482],
       [0.81131319, 1.        , 0.96578083, ..., 0.96728494, 0.93323413,
        0.95138682],
       [0.83194573, 0.96578083, 1.        , ..., 0.9668292 , 0.97331513,
        0.97052851],
       [0.79178443, 0.97585818, 0.96054717, ..., 0.99008252, 0.92960836,
        0.94061843],
       [0.96221297, 0.76508768, 0.81803765, ..., 0.81970446, 0.85794287,
        0.86929201]])

In [ ]:
subset100.head()

In [ ]:
def cos_similar_songs_playlist(cos_matrix, orig_df, target_inx, k):
    """
    cos_matrix: cosine matrix of the tracks
    orig_df: original df with tracks as rows, but with playlistid and other features (e.g., train)
    target_inx: index of playlist
    
    """
    target_playlistid = orig_df.loc[0, "Playlistid"]
    target_track_inx = np.where(orig_df["Playlistid"] == target_playlistid)
    # remove rows in matrix with songs that live in the same playlist with the target song
    # candidate_cos_matrix = np.delete(cos_matrix, target_track_inx, 1) # remoe songs that co-exist in the same playlist

    candidate_cos_matrix = cos_matrix
    candidate_song_rec = candidate_cos_matrix[target_inx, ] # row of target playlist
    candidate_song_rec_inx = np.argsort(candidate_song_rec) # get index of sorted cosine similarity

    candidate_song_rec[candidate_song_rec_inx][::-1]
    ## Todo: sort the row with highest (recommend most similar tracks)

    target_playlistid = train

In [16]:
train.head()

,Playlistid,Trackid,Artist_Name,Track_Name,Album_Name,Track_Duration,Artist_uri,Track_uri,Album_uri,acousticness,...,loudness,mode,speechiness,tempo,time_signature,valence,Playlist,Album,Track,Artist
830,61388,68,Drake,"Hold On, We're Going Home",Nothing Was The Same,227880,spotify:artist:3TVXtAsR1Inumwj472S9r4,spotify:track:6jdOi5U5LBzQrc4c1VT983,spotify:album:2ZUFSbIkmFkGag000RWOpA,0.00357,...,-7.438,0,0.0949,99.999,4,0.287,OVO,39,95,18
667,51590,28,T-Pain,Buy U a Drank (Shawty Snappin'),Epiphany,227960,spotify:artist:3aQeKQSyrW4qWr35idm0cy,spotify:track:4gFXY5yvHayGckJndafYQE,spotify:album:1fXu5FuLGSpQdmG5SWJfXL,0.01110,...,-8.127,1,0.1140,80.028,4,0.612,Rap,30,38,23
1842,193450,1,Hailee Steinfeld,Starving,HAIZ,181933,spotify:artist:5p7f24Rk5HkUZsaS3BLG5F,spotify:track:6875MeXyCW0wLyT72Eetmo,spotify:album:5m01tty1948HnkXUhG0urw,0.41400,...,-4.225,1,0.1020,99.852,4,0.498,Fall 2016,51,55,45
2005,198885,26,X Ambassadors,Renegades,VHS,195199,spotify:artist:3NPpFNZtSTHheNBaWC82rB,spotify:track:46GGxd8TVRt6FjUBfCavVT,spotify:album:5yQ3ctYzF5d1SgMzhyWe8s,0.01270,...,-5.995,1,0.0896,89.833,4,0.600,my favorite music,22,27,18
934,73524,1,Justin Timberlake,Suit & Tie,The 20/20 Experience (Deluxe Version),326280,spotify:artist:31TPClRtHm23RisEBtV3X7,spotify:track:6vt0I1cw1YmAIKDJvHVIM5,spotify:album:0O82niJ0NpcptYRxogeEZu,0.10300,...,-3.799,0,0.2190,101.985,4,0.357,Barry,6,8,4


In [18]:
cos_matrix = train_scaled_cos_matrix
target_inx = 0 # target playlist inx
## Above should be removed
target_playlistid = train.loc[0, "Playlistid"]

target_track_inx = np.where(train["Playlistid"] == target_playlistid)
# remove rows in matrix with songs that live in the same playlist with the target song
# candidate_cos_matrix = np.delete(cos_matrix, target_track_inx, 1) # remoe songs that co-exist in the same playlist

candidate_cos_matrix = cos_matrix
candidate_song_rec = candidate_cos_matrix[target_inx, ] # row of target playlist
candidate_song_rec_inx = np.argsort(candidate_song_rec) # get index of sorted cosine similarity

candidate_song_rec[candidate_song_rec_inx][::-1]
## Todo: sort the row with highest (recommend most similar tracks)

array([1.        , 1.        , 1.        , ..., 0.54621412, 0.52038527,
       0.35889349])

In [19]:
candidate_song_rec_inx.shape

(1970,)

In [22]:
train['Track_Name'][candidate_song_rec_inx][::-1]

0                               Hold On, We're Going Home
1414                            Hold On, We're Going Home
1289                            Hold On, We're Going Home
791                                                  Blem
1672                                        Teenage Fever
24                                              Needed Me
1299                                              Used To
236                                               Used To
1332                                               Havana
1764                                           King Kunta
320                                            King Kunta
1336                                              My Type
258                                  Don't Stop The Music
1638                                 Don't Stop The Music
1306                                             Heathens
903                                              Heathens
1092                                             Heathens
1132          

In [27]:
unique_candidate_song_sorted = train['Track_uri'][candidate_song_rec_inx][::-1].drop_duplicates()
unique_candidate_song_sorted

0       spotify:track:6jdOi5U5LBzQrc4c1VT983
791     spotify:track:2XlHu0HcujBCkWMdIAvrqt
1672    spotify:track:6n3HGiq4v35D6eFOSwqYuo
24      spotify:track:1Tt4sE4pXi57mTD1GCzsqm
1299    spotify:track:63p3Slj0shAFP0Q7dQetj5
1332    spotify:track:0ofbQMrRDsUaVKq2mGLEAb
1764    spotify:track:0N3W5peJUQtI4eyR6GJT5O
1336    spotify:track:6V9kwssTrwkKT72imgowj9
258     spotify:track:62LHRv9uwSNlBmByQF5jdE
1306    spotify:track:6i0V12jOa3mr6uu4WYhUBr
1132    spotify:track:0B8QzDH7YWih85V5SEMnyJ
389     spotify:track:7jslhIiELQkgW9IHeYNOWE
746     spotify:track:3Jrs7cC2OJNlNSy9VnDOMZ
402     spotify:track:7givkGZW1cwM8MJYzMd2Na
667     spotify:track:3CJvmtWw2bJsudbAC5uCQk
255     spotify:track:1X2Zd5wKGbY1oKzb8dzJRy
706     spotify:track:7tGlzXJv6GD5e5qlu5YmDg
1273    spotify:track:07dYGGSrzPeg6a3KZjWX65
721     spotify:track:6gcxng9dLaBV9EHNUDfYX1
1280    spotify:track:5Sf3GyLEAzJXxZ5mbCPXTu
1884    spotify:track:02XnQdf7sipaKBBHixz3Zp
263     spotify:track:0ESJlaM8CE1jRWaNtwSNj8
758     sp

In [36]:
tracks_in_target_playlist = train.loc[train["Playlistid"] == target_playlistid, "Track_uri"]

In [45]:
song_to_recommend = np.array(unique_candidate_song_sorted.loc[~unique_candidate_song_sorted.isin(tracks_in_target_playlist) ])

In [46]:
song_to_recommend[0]

'spotify:track:1Tt4sE4pXi57mTD1GCzsqm'

In [ ]:
candidate_song_rec[candidate_song_rec_inx][::-1]

In [ ]:
candidate_song_rec_inx.shape

In [ ]:
subset100.shape

In [ ]:
train.head()

In [ ]:
target_track_inx

In [ ]:
candidate_cos_matrix

In [ ]:
candidate_song_rec

In [ ]:
candidate_song_rec[candidate_song_rec_inx][::-1]

In [ ]:
x = np.array([3, 1, 2])
inx_trial = np.argsort(x)
# array([1, 2, 0])

In [ ]:
inx_trial # 1st item should be at third position

In [ ]:
x[np.array([0,1])]

In [ ]:
x[inx_trial][::-1]

In [ ]:
candidate_cos_matrix[0,]

In [ ]:
candidate_song_rec

In [ ]:
cos_matrix[target_inx,]

In [ ]:
cos_matrix[:5]

In [ ]:
target_track_inx

In [ ]:
candidate_cos_matrix = np.delete(cos_matrix, target_track_inx, 0)

In [ ]:
candidate_cos_matrix

In [ ]:
candidate_cos_matrix = cos_matrix[]

In [ ]:
train.head()

In [ ]:
# target_playlistid
train["Playlistid"]

In [ ]:
# Create Binary Sparse Matrix
co_mat = pd.crosstab(train.Playlistid, train.Track_uri)
co_mat = co_mat.clip(upper=1)
assert np.max(co_mat.describe().loc['max']) == 1

co_mat_sparse = csr_matrix(co_mat)

In [ ]:
# Train kNN model
col_filter = NearestNeighbors(metric='cosine', algorithm='brute')
col_filter.fit(co_mat_sparse)

## Making Predictions

In [ ]:
def nholdout(playlist_id, df):
    '''Pass in a playlist id to get number of songs held out in val/test set'''
    
    return len(df[df.Playlistid == playlist_id].Track_uri)

In [ ]:
def kpredict(knnmodel, playlist_id, df):
    '''for a playlist id, generate list of 15*k predictions where k is num holdouts''' 
    
    k = nholdout(playlist_id, df)*15 # number of holdouts
    ref_songs = co_mat.columns.values[co_mat.loc[playlist_id] == 1] # songs already in playlist
    dist, ind = knnmodel.kneighbors(np.array(co_mat.loc[playlist_id]).reshape(1, -1), n_neighbors = 99)
    rec_ind = co_mat.index[ind[0]] # recommended playlists
    
    n_pred = 0
    pred = []
    for i in rec_ind:
        new_songs = co_mat.columns.values[co_mat.loc[i] == 1] # potential recommendations
        for song in new_songs:
            if song not in ref_songs: # only getting songs not already in target playlist
                pred.append(song)
                n_pred += 1
                if n_pred == k:
                    break
        if n_pred == k:
            break
    
    return pred

In [ ]:
### Prediction Example
pi = 430 # target playlist index
kpreds = kpredict(col_filter, pi, val) # list of predictions

In [ ]:
val_set = val[val.Playlistid == pi]
val_set = val_set['Track_uri'] # ground truth

## Metrics

In [ ]:
def r_precision(prediction, val_set):
# prediction should be a list of predictions
# val_set should be pandas Series of ground truths
    score = np.sum(val_set.isin(prediction))/val_set.shape[0]
    return score

In [ ]:
### Example Usage
r_precision(kpreds, val_set)

In [ ]:
### NDCG Code Source: https://gist.github.com/bwhite/3726239
def dcg_at_k(r, k, method=0):
    r = np.asfarray(r)[:k]
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.


def ndcg_at_k(r, k, method=0):
    dcg_max = dcg_at_k(sorted(r, reverse=True), k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max

In [ ]:
### Example Usage
# Generate binary relevance array
r = np.zeros(len(kpreds))
for i, p in enumerate(kpreds):
    if p in val_set:
        r[i] = 1

ndcg_at_k(r, len(r))

## Baseline Model Performance

In [ ]:
rps = []
ndcgs = []
for pid in co_mat.index:
    ps = kpredict(col_filter, pid, val) # predictions
    vs = val[val.Playlistid == pid].Track_uri # ground truth
    rps.append(r_precision(ps, vs))
    
    r = np.zeros(len(ps))
    for i, p in enumerate(ps):
        if np.any(vs.isin([p])):
            r[i] = 1
    ndcgs.append(ndcg_at_k(r, len(r)))
    

In [ ]:
avg_rp = np.mean(rps)
avg_ndcg = np.mean(ndcgs)
print('Avg. R-Precision: ', avg_rp)
print('Avg. NDCG: ', avg_ndcg)
print('Total Sum: ', np.mean([avg_rp, avg_ndcg]))